In [589]:
import logging
import pyprind
import sqlite3
import re
import urllib3
from bs4 import BeautifulSoup
import requests
import pandas as pd
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [590]:
logger = logging.getLogger('cricket')
logger.setLevel(logging.DEBUG)

## Create match_table

In [505]:
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute('SELECT COUNT(*) FROM {tn}'.\
        format(tn=table_player_list))
print c.fetchall()[0]
conn.close()

(2850,)


## Create a player-match table

In [591]:
class MySoup(object):
    __http = None 
    def __init__(self):
        self.__http  = urllib3.PoolManager()
    
    def get_soup(self, url):
        if not self.__http:
            logger.critical("urllib3.PoolManager() not initialized!")
        response = self.__http.request('GET', url)
        return BeautifulSoup(response.data)

In [592]:
def get_list_of_urls_for_players_per_their_first_char():
    # Example: 
    # url_list[0] corresponds to the page with players name starting with A 
    # url_list[1] corresponds to the page with players name starting with B
    base_url = 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group='
    url_list = [base_url + chr(ord('A') + idx) for idx in range(26)] 
    return url_list


def get_player_code(plr_att, match_type="test"):
    TEST_COLUMN = 3
    ODI_COLUMN = 4
    T20_COLUMN = 5
    
    d = {"test": TEST_COLUMN, "odi": ODI_COLUMN, "t20": T20_COLUMN}
    
    if d[match_type] >= len(plr_att):
        return None
        
    player_col = plr_att[d[match_type]].find('a')
    if player_col:
        url = player_col.get('href')
        m = re.search("PlayerID=(.*)", url)
        return m.groups()[0] 
    else:
        return None

def get_player_name(plr_attr):
    name = plr_attr[0].find(text=True).strip()
    name = name.replace("'", "")
    return name
    
def get_dob(plr_attr):
    date_string = plr_attr[1].find(text=True).strip()
    date = [token.strip() for token in date_string.split('/')]
    year = date[2]
    month = date[1]
    day = date[0]
    date = int(year + month + day)
    return date

def get_country(plr_attr):
    return plr_attr[2].find(text=True).strip()

In [593]:
def get_list_of_urls_for_test_matches_per_year(match_type="test"):
    # From match_list_menu_url, we find all links for the pages which 
    # contain matches played in a particular year (or a period of years).
    
    match_list_menu_url = 'http://www.howstat.com/cricket/Statistics/Matches/MatchListMenu.asp?r=T#tests?data-scroll=false'
    my_soup = MySoup()
    soup = my_soup.get_soup(match_list_menu_url)
    
    yearwise_test_match_list = []
    match_list_per_year_url_base = "http://www.howstat.com/cricket/Statistics/Matches/MatchList.asp?Group="
    list_of_urls = soup.find_all('a', { "class" : "LinkOff" })
    for url in list_of_urls:
        ref = url.get('href')
        if match_type!="test":
            return yearwise_test_match_list
        m = re.search("MatchList\.asp\?Group=(.*)", ref) # For test matches
        if m is not None:
            yearwise_test_match_list.append(match_list_per_year_url_base + m.groups()[0])
    return yearwise_test_match_list 

def get_date(date_string):
    date = [token.strip() for token in date_string.split('/')]
    year = date[2]
    month = date[1]
    day = date[0]
    date = int(year + month + day)
    return date

def get_countries(countries_string):
    countries = countries_string.strip()
    countries = [token.strip() for token in countries.split("v.")]
    return (countries[0], countries[1])

def get_ground_code(ground):
    m = re.search("GroundCode=(.*)", ground)
    ground_code = m.groups()[0].strip()
    return ground_code

def get_result(res_attribute, country_1, country_2):
    if "Drawn" in res_attribute.find(text=True):
        return "Match Drawn"
    elif "drawn" in res_attribute.find(text=True):
        return "Match Drawn"
    elif "abandoned" in res_attribute.find(text=True):
        return "Match Abandoned"
    elif "Abandoned" in res_attribute.find(text=True):
        return "Match Abandoned"
    
    if res_attribute.find('span') is None:
        return "Match Abandoned"
    
    result = res_attribute.find('span').find(text=True)
    if (country_1 in result):
        return country_1
    elif (country_2 in result):
        return country_2
    else:
        return "Match Drawn"
    
def get_match_code(scorecard):
    m = re.search("MatchCode=(.*)", scorecard)
    match_code = m.groups()[0].strip()
    return match_code

In [594]:
def get_toss_winning_country(soup):
    table = soup.find('table')
    tables = table.find_all('table')
    toss_table_idx = 4
    toss_row_idx = 3
    table_rows = tables[toss_table_idx].find_all('tr')
    table_data = table_rows[toss_row_idx].find_all('td')
    return table_data[1].text.strip()

def getPlayerId(url):
    m = re.search('PlayerID=(.*)', url)
    if m is None:
        return None
    return m.groups()[0]

def getPlayerCountry(playerId):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    c.execute("SELECT country FROM {tn} where player_code='{pid}'".\
              format(tn=table_player_list, pid=playerId))
    country = c.fetchall()[0][0]
    conn.close()
    return country

def get_country_1(player_country):
    return player_country[0][1]
    
def get_country_2(player_country):
    return player_country[11][1]
    
def get_ordered_batsmen_country_1(player_country):
    return player_country[0:11]

def get_ordered_batsmen_country_2(player_country, country_2):
    country_2_bowler_idx_start = 11
    country_2_batsman_idx_end = country_2_bowler_idx_start
    while (country_2_batsman_idx_end + 1 < len(player_country) and player_country[country_2_batsman_idx_end + 1][1] == country_2) :
        country_2_batsman_idx_end += 1

    country_2_batsmen_idx_start = country_2_batsman_idx_end - 10 
    return player_country[country_2_batsmen_idx_start:country_2_batsman_idx_end + 1]
    
def get_ordered_bowler_country_2(player_country, country_2):
    country_2_bowler_idx_start = 11
    country_2_batsman_idx_end = country_2_bowler_idx_start
    while (player_country[country_2_batsman_idx_end + 1][1] == country_2):
        country_2_batsman_idx_end += 1
    country_2_bowler_idx_end = country_2_batsman_idx_end - 11
    return player_country[country_2_bowler_idx_start:country_2_bowler_idx_end + 1]

def get_ordered_bowler_country_1(player_country, country_1):
    country_1_batting_idx_end = 10
    country_2_batting_idx_end = country_1_batting_idx_end
    while (player_country[country_2_batting_idx_end + 1][1] != country_1):
        country_2_batting_idx_end += 1
        
    country_1_bowling_idx_start = country_2_batting_idx_end + 1
    country_1_batting_idx_end = country_1_bowling_idx_start
    while (player_country[country_1_batting_idx_end + 1][1] == country_1):
        country_1_batting_idx_end += 1
    country_1_bowling_idx_end = country_1_batting_idx_end - 11

    return player_country[country_1_bowling_idx_start: country_1_bowling_idx_end + 1]

In [614]:
class CricketDB(object):
    sqlite_file = ""
    table_match_summary = ""
    table_player_list = ""
    table_player_stats_per_match = ""
    __conn = None
    __cursor = None
    
    def __init__(self, 
                 sqlite_file, 
                 table_match_summary, 
                 table_player_list, 
                 table_player_stats_per_match):
        self.sqlite_file = sqlite_file
        self.table_match_summary = table_match_summary
        self.table_player_list = table_player_list
        self.table_player_stats_per_match = table_player_stats_per_match
        
        self.open_connection()
        self.create_tables()
        
    def open_connection(self):
        if not self.is_connection_open():
            self.__conn = sqlite3.connect(self.sqlite_file)
        self.__cursor = self.__conn.cursor()
        
    def close_connection(self):
        self.__conn.close()
        
    def commit_changes(self):
        self.__conn.commit()
        
    def commit_and_close(self):
        self.__conn.commit()
        self.__conn.close()
        
    def is_connection_open(self):
        try:
            # The following is a proxy to check whether the connection is open
            # Did not find a connection status for sqlite3 for python! 
            self.__cursor.execute('select name from sqlite_master where type = "table";')
        except:
            logger.info("Connection is closed")
            return False
        else:
            print self.__cursor.fetchall()
            logger.info("Connection is open")
            return True   
    
    def get_existing_tables(self):
        tables = []
        try:
            self.__cursor.execute('select name from sqlite_master where type = "table";')
            tables = [tables_tup[0] for tables_tup in self.__cursor.fetchall()]  
        finally:
            return tables
            
    def create_tables(self):
        tables = self.get_existing_tables()
        if self.table_player_list not in tables:
            self.create_table_player_list()
        if self.table_match_summary not in tables:
            self.create_table_match_summary()
        if self.table_player_stats_per_match not in tables:
            self.create_table_player_stats_per_match()
    
    def create_table_player_list(self):
        if not self.is_connection_open():
            self.open_connection()
            
        self.__cursor.execute('CREATE TABLE {tn} (player_code TEXT PRIMARY KEY, name TEXT, dob INTEGER, country TEXT)'\
                              .format(tn=table_player_list))
        self.commit_and_close()
    
    def create_table_match_summary(self):
        if not self.is_connection_open():
            self.open_connection()
        
        self.__cursor.execute('CREATE TABLE {tn}\
        (match_code TEXT PRIMARY KEY, \
        date INTEGER, \
        country_1 TEXT, \
        country_2 TEXT, \
        host TEXT, \
        ground_code TEXT, \
        toss_winning_country TEXT, \
        result TEXT)'.format(tn=self.table_match_summary))
        
        countries = ['ct'+str(idx) for idx in range(1,3)]
        players = ['pl'+str(idx) for idx in range(1,12)]
        for country in countries:
            for player in players:
                column_name = country + "_" + player
                self.__cursor.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}"\
                                      .format(tn=self.table_match_summary, cn=column_name, ct='TEXT'))
        self.commit_and_close()    
    
    def create_table_player_stats_per_match(self):
        if not self.is_connection_open():
            self.open_connection()
        
        self.__cursor.execute('CREATE TABLE {tn} (match_code TEXT, player_code TEXT, \
        total_runs INTEGER DEFAULT 0, \
        batting_avg REAL, \
        total_wickets INTEGER DEFAULT 0, \
        bowling_avg REAL, \
        FOREIGN KEY(match_code) REFERENCES match_table(match_code), \
        FOREIGN KEY(player_code) REFERENCES player_list_table(player_code))'.format(tn=self.table_player_stats_per_match))   
        self.commit_and_close()      
        
    def __update_players_list(self, list_of_urls_for_players_per_their_first_char, match_type):
        self.open_connection()
        my_soup = MySoup()
        for url in list_of_urls_for_players_per_their_first_char:
            soup = my_soup.get_soup(url)
            tables = soup.find_all('table', attrs={'class':'TableLined'})
            table = tables[0] # there is only one table per page
            
            count = 0
            for table_row in table.find_all('tr'):
                if count < 2: # The first two rows are not player information
                    count += 1
                    continue
                player_attributes = table_row.find_all('td')
                player_code = get_player_code(player_attributes, match_type)
                if player_code is None:
                    continue
                player_name = get_player_name(player_attributes)
                dob = get_dob(player_attributes)
                country = get_country(player_attributes)
                
                
                self.__cursor.execute("INSERT INTO {tn} \
                                      (player_code, name, dob, country) \
                                      VALUES ('{pc}', '{nm}', {dt}, '{cy}')"\
                                      .format(tn=table_player_list, pc=player_code, nm=player_name, dt=dob, cy=country))
        self.commit_and_close()
        logger.debug("Done updating player list")
 

    def __update_match_summary(self, list_of_urls_for_test_matches_per_year, match_type):
        self.open_connection()
        my_soup = MySoup()
        
        for url in list_of_urls_for_test_matches_per_year:
            soup = my_soup.get_soup(url)
        
            tables = soup.find_all('table', attrs={'class':'TableLined'})
            table = tables[0] # there is only one table per page
            match_rows = table.find_all('tr')
        
            count = 0
            for match_row in match_rows:
                if count is 0:
                    # The forst row is the header row which does not have match info
                    count += 1
                    continue
            
                # get match attributes
                match_attributes = match_row.find_all('td')    
                #print match_attributes
                date = get_date(match_attributes[1].find(text=True)) 
                country_1, country_2 = get_countries(match_attributes[2].find('a').find(text=True))
                ground_code = get_ground_code(match_attributes[3].find('a').get('href'))
                result = get_result(match_attributes[4], country_1, country_2)
                match_code = get_match_code(match_attributes[5].find('a').get('href'))
                toss_winning_country = " "  # Need to get this
                if match_type == "test":
                    host = country_1
                else:
                    host = " "
                
                self.__cursor.execute("INSERT INTO {tn} (match_code, date, country_1, \
                country_2, host, ground_code, toss_winning_country, result) \
                VALUES ('{mc}', {dt}, '{c1}', '{c2}', '{ht}', '{gc}', '{ts}', '{rt}')"\
                          .format(tn=table_match_summary, mc=match_code, \
                                                dt=date, c1=country_1, c2=country_2, \
                                                ht=host, gc=ground_code, ts=toss_winning_country, rt=result))

        self.commit_and_close()
        print "Done"
        
    def __get_list_of_match_urls(self):
        self.open_connection()
        self.__cursor.execute("SELECT match_code FROM match_table")
        base_url = 'http://www.howstat.com/cricket/Statistics/Matches/MatchScorecard.asp?MatchCode='
        urls = [base_url + m_c[0] for m_c in self.__cursor.fetchall()]
        self.close_connection()
        return urls

    def __update_match_summary_with_player_data(self, list_of_match_urls):
        self.open_connection()
        my_soup = MySoup()
        missing_data_match_codes = []
        
        pbar = pyprind.ProgBar(len(list_of_match_urls))
        for url in list_of_match_urls:
            m = re.search("MatchCode=(.*)", url)
            match_code = m.groups()[0]
            
            #print match_code
            soup = my_soup.get_soup(url)
        
            player_country = []
            for links_data in soup.find_all('a', { "class" : "LinkOff" }):
                player_url = links_data.get('href')
                pid = getPlayerId(player_url)    
                if pid is not None:
                    country = getPlayerCountry(pid)
                    player_country.append((pid,country))
                
            if len(player_country) < 22:
                missing_data_match_codes.append(match_code)
                print "missing data for match_code = {mc}".format(mc=match_code)
                continue
                
            country_1 = get_country_1(player_country)
            country_2 = get_country_2(player_country)
            ordered_batsmen_country_1 = get_ordered_batsmen_country_1(player_country)
            ordered_batsmen_country_2 = get_ordered_batsmen_country_2(player_country, country_2) 
            #ordered_bowler_country_2 = get_ordered_bowler_country_2(player_country, country_2) 
            #ordered_bowler_country_1 = get_ordered_bowler_country_1(player_country, country_1) 
        
            toss_winning_country = get_toss_winning_country(soup)
          
            query = "UPDATE {tn} SET country_1='{c1}', country_2='{c2}', toss_winning_country='{tc}'"
            for pid in range(1, 12):
                query = query + ", ct1"+"_pl"+str(pid) + "='" + ordered_batsmen_country_1[pid-1][0] + "'"
            for pid in range(1, 12):
                query = query + ", ct2"+"_pl"+str(pid) + "='" + ordered_batsmen_country_2[pid-1][0] + "'"
            query = query + "  WHERE match_code='{mc}'"
            
            self.__cursor.execute(query.format(tn=table_match_summary, c1=country_1, c2=country_2, tc=toss_winning_country, mc=match_code))
            self.commit_changes()
            pbar.update()
            
        self.close_connection()
        return missing_data_match_codes
        
        
    def populate_table_player_list(self):
        match_type = "test"
        urls_for_players_per_their_first_char = get_list_of_urls_for_players_per_their_first_char()
        self.__update_players_list(urls_for_players_per_their_first_char, match_type)
        
    def populate_table_match_summary(self):
        match_type = "test"
        list_of_urls_for_test_matches_per_year = get_list_of_urls_for_test_matches_per_year(match_type) # UNCOMMENT
        self.__update_match_summary(list_of_urls_for_test_matches_per_year, match_type) # UNCOMMENT
        list_of_match_urls =  self.__get_list_of_match_urls()
        missing_data_match_codes = self.__update_match_summary_with_player_data(list_of_match_urls)
        print "There are {l} matches with missing data".format(l=len(missing_data_match_codes))
        print missing_data_match_codes
        
    def populate_table_player_stats_per_match(self):
        return 

In [615]:
def main():
    
    sqlite_file = '/home/shantanu/PycharmProjects/cricketResultPrediction/CricketDB.sqlite'
    table_match_summary = 'match_table'
    table_player_list = "player_list_table"
    table_player_stats_per_match = 'player_stats_per_match_table'
    db = CricketDB(sqlite_file, table_match_summary, table_player_list, table_player_stats_per_match)
    
    #db.populate_table_player_list()
    #db.populate_table_match_summary()
    db.populate_table_player_stats_per_match()
    
    
if __name__=="__main__":
    main()

[(u'match_table',), (u'player_list_table',)]


In [605]:
missing_data_match_codes = ['1266', '1267', '1268', '1269', '1270', '1271', '1272', '1273', '1276', '1447', '1448', '1619']